# M5 Model

This notebook **heavily** references the tutorial: *Speech Command Classification with torchaudio*.
- [Tutorial](https://pytorch.org/tutorials/intermediate/speech_command_classification_with_torchaudio_tutorial.html)

## Derivations
We simplified the tutorial by:
- Wrapping the PyTorch model in PyTorch-Lightning, abstracting away all low-level nitty-gritty loops
- The M5 Model is simplified in code by wrapping common blocks to avoid redundant code: See `cnn_m5.py`
- Using PyTorch-Lightning `Trainer`, we limit the number of batches, to make it run quicker

## M5 Model Reference
[Research Paper](https://arxiv.org/pdf/1610.00087.pdf)
Dai, Wei, et al. "Very deep convolutional neural networks for raw waveforms." 2017 IEEE international conference on acoustics, speech and signal processing (ICASSP). IEEE, 2017.

## Results
Through these settings below, we're able to yield 77% accuracy after 5 epochs, see the TensorBoard for history.
The model has not reach states of overfit, instead, was cut short as it takes a long time.

In [2]:
import pytorch_lightning as pl
import torch
from torchaudio.transforms import Vad

from src.model.cnn_mel import CNNMel
from src.model.lit_wrapper import LitWrapper
from src.speech_command_dataset import SpeechCommandDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} Backend!")

ds = SpeechCommandDataset(batch_size=128, num_workers=0)  #, dl_kwargs={'pin_memory': True})
train_dl, val_dl, test_dl = ds.dls()

model = LitWrapper(CNNMel(len(ds.classes), n_time=81, n_channel=80), ds.classes, lr=0.01)

trainer = pl.Trainer(
    default_root_dir="cnn_mel/",
    max_epochs=5,
    limit_train_batches=192,
    limit_val_batches=32,
    limit_predict_batches=1,
    # profiler='simple'
    # fast_dev_run=True
)

trainer.fit(model, train_dataloaders=train_dl, val_dataloaders=val_dl)
# pred = trainer.predict(model, dataloaders=test_dl)

Using cuda Backend!


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\johnc\anaconda3\envs\snn_voice\lib\site-packages\pytorch_lightning\trainer\trainer.py:1764: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.

  | Name  | Type   | Params
---------------------------------
0 | model | CNNMel | 49.9 K
---------------------------------
49.9 K    Trainable params
0         Non-trainable params
49.9 K    Total params
0.199     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\johnc\anaconda3\envs\snn_voice\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\johnc\anaconda3\envs\snn_voice\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

C:\Users\johnc\anaconda3\envs\snn_voice\lib\site-packages\pytorch_lightning\trainer\trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
